In [1]:
import numpy as np
import random
from sklearn.mixture import BayesianGaussianMixture as DPGMM
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import multivariate_normal
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors, animation
from IPython.display import HTML

In [22]:
def initialize_figure():
    fig, axes = plt.subplots(1,3)
    axes = axes.flatten()
    fig.set_size_inches(15,5)
    for ax in axes:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    #plt.axis('equal')
    axes[0].set_title("Samples", fontsize=18)
    plt.tight_layout()
    return fig, axes

def get_data(num_dim_pts=20, neg_start=0, neg_end=6, xoffset=6, yoffset=6):
    pos = None; neg = None
    for x in range(num_dim_pts):
        for y in range(num_dim_pts):
            inst = np.array([x,y]).reshape(1,-1)
            if (x >= xoffset+neg_start and x < xoffset+neg_end and
                y >= yoffset+neg_start and y < yoffset+neg_end):
                neg = inst if neg is None else np.vstack((neg, inst))
            else:
                pos = inst if pos is None else np.vstack((pos, inst))
    # Prepend labels
    pos = np.insert(pos, 0, 1, axis=1)
    neg = np.insert(neg, 0, 0, axis=1)
    return np.vstack((pos, neg))

def get_grid_data(data, num_dim_pts=20):
    grid = np.zeros((num_dim_pts, num_dim_pts))
    for point, value in data:
        point = point.flatten()
        grid[point[1], point[2]] = value
    return grid.T

def plot_scatter_data(data, ax, cname, large=False):
    converter = colors.ColorConverter()
    color = converter.to_rgba(colors.cnames[cname])
    if large:
        scatter = ax.scatter(data[1], data[2], color=color, s=300.0, edgecolor='black', linewidth='3')
    else:
        scatter = ax.scatter(data[:,0], data[:,1], color=color, s=100.0)
    return scatter
    
def plot_gmm(samples, gmm, ax, cname):
    preds = gmm.predict(samples)
    means = gmm.means_
    covs  = gmm.covariances_
    color = colors.cnames[cname]
    for i, (mean, cov) in enumerate(zip(means, covs)):
        v, w = np.linalg.eigh(cov)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / np.linalg.norm(w[0])
        # Not plotting redundant components
        if not np.any(preds == i):
            continue
        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        
def plot_grid_img(data, ax, title="", vmin=0.0, vmax=0.7):
    ax.set_title(title, fontsize=18)
    num_dim_pts = 20
    grid = np.zeros((num_dim_pts, num_dim_pts))
    for point, value in data:
        point = point.flatten()
        grid[point[1], point[2]] = value
    img = ax.imshow(grid.T, origin="lower", vmin=vmin, vmax=vmax, cmap="hot")
    return img

def plot_line(data, ax, lw=5.0, color='g', title="Total Entropy"):
    ax.plot(data)
    ax.set_ylim(0,0.7)
    ax.set_xlim(0,100)
    ax.set_title(title, fontsize=18)
        
def pdf(x, mu, sigma):
    try:
        mvn = multivariate_normal(mean=mu, cov=sigma)
        return mvn.pdf(x)
    except np.linalg.LinAlgError as e:
        print "SINGULAR IN TRAJ PROB"
        return 0.0
    
def gmm_pdf(x, gmm, zero_lim=1e-3):
    p = 0.0
    for i in range(gmm.n_components):
        p += gmm.weights_[i] * pdf(x, gmm.means_[i], gmm.covariances_[i])
    return max(p, zero_lim)

def get_probs(data, pos_gmm, neg_gmm):
    pos_probs = []
    neg_probs = []
    for i in range(data.shape[0]):
        x = data[i,1:] # Leave out pre-pended label
        pos_pdf  = gmm_pdf(x, pos_gmm)
        neg_pdf  = gmm_pdf(x, neg_gmm)
        pos_prob = pos_pdf / (pos_pdf + neg_pdf)
        neg_prob = neg_pdf / (pos_pdf + neg_pdf)
        pos_probs.append((data[i,:], pos_prob))
        neg_probs.append((data[i,:], neg_prob))
    return pos_probs, neg_probs

def entropy_measure(pos_prob, neg_prob):
    pos_entropy = -(pos_prob * np.log(pos_prob))
    neg_entropy = -(neg_prob * np.log(neg_prob))
    return pos_entropy + neg_entropy

def margin_measure(pos_prob, neg_prob):
    # Only two classes, so this amounts to absval
    return abs(pos_prob - neg_prob)

def get_measures(pos_probs, neg_probs, measure_f=entropy_measure):
    measures = []
    for i in range(len(pos_probs)):
        m = measure_f(pos_probs[i][1], neg_probs[i][1])
        measures.append((pos_probs[i][0], m))
    return measures

In [23]:
dp_prior   = True
num_frames = 5
us_method  = "Entropy"  # ["Entropy", "Random"]

# ===================================================================================

if dp_prior:
    pos_gmm = DPGMM()
    neg_gmm = DPGMM()
else:
    pos_gmm = GMM()
    neg_gmm = GMM()
    
data = get_data()
full_data = np.copy(data)
pos_data = data[np.where(data[:,0] == 1)][:,1:]
neg_data = data[np.where(data[:,0] == 0)][:,1:]
fig, axes = initialize_figure()

# Get two samples of each type to initiate learning
pos_samples = pos_data[np.random.randint(pos_data.shape[0], size=3),:]
neg_samples = neg_data[np.random.randint(neg_data.shape[0], size=3),:]

# Fit the GMMs
pos_gmm.fit(pos_samples)
neg_gmm.fit(neg_samples)

# Get initial probabilities and measures
pos_probs, neg_probs = get_probs(data, pos_gmm, neg_gmm)
measures = get_measures(pos_probs, neg_probs)

# Setup the plots
scatter = None
tot_entropies = []
plot_grid_img(measures, axes[1], "Entropy")
plot_line(tot_entropies, axes[2])


def iterate(itr):
    
    global pos_samples, neg_samples, data, tot_entropies, scatter
    
    # Clear drawn GMMs
    axes[0].artists = []
    axes[0].set_ylim(-1,20)
    axes[0].set_xlim(-1,20)
    
    if scatter is not None:
        scatter.remove()
    
    # Visualize GMMs before updating since probs in this iteration are computed based on these ones
    plot_gmm(pos_samples, pos_gmm, axes[0], "cornflowerblue")
    plot_gmm(neg_samples, neg_gmm, axes[0], "firebrick")
    
    # Compute probs/measures over all data for visualization
    full_pos_probs, full_neg_probs = get_probs(full_data, pos_gmm, neg_gmm)
    full_measures = get_measures(full_pos_probs, full_neg_probs)
    
    pos_probs, neg_probs = get_probs(data, pos_gmm, neg_gmm)
    if us_method == "Random":
        measures = get_measures(pos_probs, neg_probs, measure_f=entropy_measure) # For tracking values
        idx = np.random.randint(0, len(measures))
        _, measure = max(measures, key=lambda p: p[1]) # Still want to max measure over all points
        next_sample = measures[idx][0]
    elif us_method == "Entropy":
        measures = get_measures(pos_probs, neg_probs, measure_f=entropy_measure)
        # Find idx of max value, collect set of all max vals (ties), pick a random one of those
        measure = 0.0
        for j in range(len(measures)):
            if measures[j][1] > measure:
                measure = measures[j][1]
        max_idxs = [k for k, value in enumerate(measures) if value[1] == measure]
        idx = np.random.choice(max_idxs)
        next_sample, measure = measures[idx]
    else:
        print "Unknown uncertainty sampling method: %s" % us_method
        return
       
    tot_entropy = 0.0
    for i in range(len(full_measures)):
        tot_entropy += full_measures[i][1]
    tot_entropies.append(tot_entropy)
    print "TOT ENTROPY", tot_entropy
    
    data = np.delete(data, idx, axis=0)
    if next_sample[0] == 1:
        pos_samples = np.vstack((pos_samples, next_sample[1:]))
    else:
        neg_samples = np.vstack((neg_samples, next_sample[1:]))
    pos_gmm.n_components = min(10, pos_samples.shape[0])
    neg_gmm.n_components = min(10, neg_samples.shape[0])
    pos_gmm.fit(pos_samples)
    neg_gmm.fit(neg_samples)

    # Visualize scatter plot
    plot_scatter_data(pos_samples, axes[0], "cornflowerblue")
    plot_scatter_data(neg_samples, axes[0], "firebrick")
    scatter = plot_scatter_data(next_sample, axes[0], "gold", large=True)
    
    # Visualize current map entropy
    plot_grid_img(full_measures, axes[1], us_method)
    
    # Visualize total map entropy over learning iterations
    plot_line(tot_entropies, axes[2])
    
ani = animation.FuncAnimation(fig, iterate, frames=num_frames)
Writer = animation.writers['ffmpeg']
writer = Writer(fps=5, bitrate=1800)
ani.save('video.mp4', writer=writer)

TOT ENTROPY 248.802788907
TOT ENTROPY 257.643305176
TOT ENTROPY 257.523481409
TOT ENTROPY 257.208105477
TOT ENTROPY 254.894571881
TOT ENTROPY 253.997764563
